In [1]:
!pip install faiss-cpu sentence-transformers --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 56.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 37.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 89.1 MB/s eta 0:00:00


### Load Model

In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
model

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

### Load & Embed Dataset

In [3]:
import numpy as np
import faiss
from sentence_transformers import SentenceTransformer

# 1. Define facts and questions
facts = [
    "The term artificial intelligence was coined in 1956.",
    "Deep learning uses neural networks with many layers.",
    "Alan Turing is considered a founding figure of AI.",
    "AI can outperform humans at chess and Go.",
    "Machine learning is a subset of AI.",
    "AI is used in medical imaging for disease detection.",
    "Natural language processing enables computers to understand text.",
    "Reinforcement learning allows AI to learn through trial and error.",
    "GPT-4 is a large language model developed by OpenAI.",
    "AI models require large amounts of data to train effectively.",
    "Image recognition is a common application of AI.",
    "Self-driving cars use AI for perception and decision-making.",
    "AI can generate realistic images from text descriptions.",
    "Speech recognition systems like Siri use AI.",
    "Ethics in AI focuses on fairness and transparency.",
    "The Turing Test measures a machine's ability to exhibit human-like intelligence.",
    "AI is applied in financial trading to detect market patterns.",
    "Transfer learning enables AI models to adapt to new tasks.",
    "Robotics combines AI with sensors and actuators.",
    "AI bias can occur if training data is not representative."
]

questions = [
    "Who is considered the father of artificial intelligence?",
    "What technology enables computers to understand and generate language?",
    "How do self-driving cars perceive their environment?",
    "Which AI technique learns by trial and error?",
    "What does the Turing Test evaluate?"
]

In [4]:
fact_embeddings = model.encode(facts, convert_to_numpy=True)
question_embeddings = model.encode(questions, convert_to_numpy=True)

In [5]:
print("Questions Matrix Shape: ", question_embeddings.shape)
print("Facts Matrix Shape: ", fact_embeddings.shape)

Questions Matrix Shape:  (5, 384)
Facts Matrix Shape:  (20, 384)


### HNSW Implementation

In [6]:
# 3. Build the HNSW index
d = fact_embeddings.shape[1]
M = 8  # Indexing Param, Smaller M is fine for small datasets
efConstruction = 16 # Indexing Param
efSearch = 8 # Retrieval Param

index = faiss.IndexHNSWFlat(d, M)
index.hnsw.efConstruction = efConstruction
index.hnsw.efSearch = efSearch

index.add(fact_embeddings)

In [7]:
# --- 3. Query the index ---
k = 3   # Top-k neighbors to retrieve
D, I = index.search(question_embeddings, k)  # D: distances, I: indices

In [8]:
print("Distances:\n\n", D)
print("="*10)
print("Indices:\n\n", I)

Distances:

 [[0.5729287  0.6944343  1.0896059 ]
 [0.78757656 1.0334734  1.15006   ]
 [0.6082597  1.555309   1.6250639 ]
 [0.4965902  0.9963416  1.0748093 ]
 [0.5017944  1.2655437  1.4911275 ]]
Indices:

 [[ 2  0  4]
 [ 6 13 15]
 [11 10 14]
 [ 7  4 17]
 [15  2  7]]


In [9]:
# --- 4. Show results ---
for qi in range(len(questions)):
    print(f"\nQuestion #{qi+1}: ", questions[qi])
    print(f"\nResults:\n")
    for i in range(k):
        print(f"{i+1}. {facts[I[qi][i]]}")
    print("="*100)


Question #1:  Who is considered the father of artificial intelligence?

Results:

1. Alan Turing is considered a founding figure of AI.
2. The term artificial intelligence was coined in 1956.
3. Machine learning is a subset of AI.

Question #2:  What technology enables computers to understand and generate language?

Results:

1. Natural language processing enables computers to understand text.
2. Speech recognition systems like Siri use AI.
3. The Turing Test measures a machine's ability to exhibit human-like intelligence.

Question #3:  How do self-driving cars perceive their environment?

Results:

1. Self-driving cars use AI for perception and decision-making.
2. Image recognition is a common application of AI.
3. Ethics in AI focuses on fairness and transparency.

Question #4:  Which AI technique learns by trial and error?

Results:

1. Reinforcement learning allows AI to learn through trial and error.
2. Machine learning is a subset of AI.
3. Transfer learning enables AI models t

### IVF Implementation

In [10]:
# 3. Build the IVF index
d = fact_embeddings.shape[1]
nlist = 5  # Number of clusters (suitable for small datasets)
nprobe = 2 # Number of clusters to search

# The quantizer is a flat index used for cluster assignment
quantizer = faiss.IndexFlatL2(d)
index = faiss.IndexIVFFlat(quantizer, d, nlist)

# IVF requires training on your data before adding vectors
index.train(fact_embeddings)
index.add(fact_embeddings)
index.nprobe = nprobe

In [11]:
# 4. Query the index with each question and print the top 3 facts
top_k = 3

for qi, question in enumerate(questions):
    D, I = index.search(question_embeddings[qi:qi+1], top_k)
    print(f"\nQ{qi+1}: {question}")
    for rank, idx in enumerate(I[0]):
        print(f"  Fact {rank+1}: {facts[idx]} (Score: {D[0][rank]:.4f})")


Q1: Who is considered the father of artificial intelligence?
  Fact 1: Alan Turing is considered a founding figure of AI. (Score: 0.5729)
  Fact 2: The term artificial intelligence was coined in 1956. (Score: 0.6944)
  Fact 3: The Turing Test measures a machine's ability to exhibit human-like intelligence. (Score: 1.1236)

Q2: What technology enables computers to understand and generate language?
  Fact 1: Natural language processing enables computers to understand text. (Score: 0.7876)
  Fact 2: Speech recognition systems like Siri use AI. (Score: 1.0335)
  Fact 3: AI can generate realistic images from text descriptions. (Score: 1.2063)

Q3: How do self-driving cars perceive their environment?
  Fact 1: Self-driving cars use AI for perception and decision-making. (Score: 0.6083)
  Fact 2: Ethics in AI focuses on fairness and transparency. (Score: 1.6251)
  Fact 3: Robotics combines AI with sensors and actuators. (Score: 1.6526)

Q4: Which AI technique learns by trial and error?
  Fac